In [ ]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from scipy.fftpack import fft2, fftshift

def extract_frames(video_path, max_frames=50):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (128, 128))
        frames.append(resized)
        count += 1
    cap.release()
    return frames

def apply_log_gabor(image):
    image = image.astype(np.float32)
    rows, cols = image.shape
    freq = fftshift(fft2(image))
    radius = np.sqrt((np.fft.fftfreq(rows)[:, None])**2 + (np.fft.fftfreq(cols)[None, :])**2)
    radius[rows // 2, cols // 2] = 1
    lg_filter = np.exp((-(np.log(radius / 0.25))**2) / (2 * (0.55**2)))
    filtered = np.abs(fftshift(freq) * lg_filter)
    return np.array([np.mean(filtered), np.std(filtered)])

def load_dataset_log_gabor(video_paths, labels, max_frames=50):
    features, new_labels = [], []
    for i, path in enumerate(video_paths):
        if not os.path.exists(path):
            continue
        frames = extract_frames(path, max_frames)
        if len(frames) == 0:
            continue
        for frame in frames:
            feat = apply_log_gabor(frame)
            features.append(feat)
            new_labels.append(labels[i])
    return np.array(features), np.array(new_labels)

def train_decision_tree(features, labels):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, stratify=labels, random_state=42)
    clf = DecisionTreeClassifier(max_depth=10, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy with Log Gabor Features (Decision Tree): {accuracy:.4f}")
    return clf

def prepare_ffpp_dataset(base_path):
    video_paths, labels = [], []
    for label, folder in enumerate(['real', 'fake']):
        folder_path = os.path.join(base_path, folder)
        if not os.path.exists(folder_path):
            continue
        for file in os.listdir(folder_path):
            if file.endswith('.mp4'):
                video_paths.append(os.path.join(folder_path, file))
                labels.append(label)
    return video_paths, labels

base_path = "C:\\Users\\Pavan\\Downloads\\FF++"
video_paths, labels = prepare_ffpp_dataset(base_path)

try:
    features, labels = load_dataset_log_gabor(video_paths, labels)
    if len(features) > 0:
        model = train_decision_tree(features, labels)
    else:
        print("No features extracted.")
except Exception as e:
    print(f"Error: {e}")
